In [2]:
import pandas as pd
data = pd.read_csv('cukrarny_union_vz2.csv')
data_test = pd.read_csv('testovaci_data.csv')

data_test = data_test.dropna()
data_test['gps_latitude'] = data_test['latitude']
data_test['gps_longitude'] = data_test['longitude']
data_test['zip'] = data_test['zip'].replace(' ', '', regex=True)


data_test['zip'] = data_test['zip'].astype(int)

positive_data = data[data['score'] != -1]
positive_data.loc[:, 'score'] = positive_data['score'] * 2
positive_data.describe()
data_test.describe()

,Unnamed: 0,distance,latitude,longitude,id_cukrarna,zip,gps_latitude,gps_longitude
count,54.000000,54.000000,54.000000,54.00000,54.000000,54.000000,54.000000,54.00000
mean,45.481481,1.876532,49.964207,14.37977,304.148148,16149.740741,49.964207,14.37977
std,28.967608,1.134259,0.009252,0.02586,163.293278,2621.646906,0.009252,0.02586
min,1.000000,0.056080,49.946800,14.33440,74.000000,14300.000000,49.946800,14.33440
25%,20.250000,0.809539,49.956800,14.35940,271.000000,15531.000000,49.956800,14.35940
50%,48.500000,1.835160,49.966800,14.37940,271.000000,15600.000000,49.966800,14.37940
75%,68.750000,2.686747,49.971800,14.39440,278.000000,15600.000000,49.971800,14.39440
max,93.000000,4.010847,49.976800,14.45940,643.000000,25246.000000,49.976800,14.45940


In [3]:

y = positive_data.score

In [4]:
positive_features = ['gps_latitude', 'gps_longitude', 'zip']

X = positive_data[positive_features]

data_test_X = data_test[positive_features]


In [5]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.25, random_state = 0)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

our_model = DecisionTreeClassifier(random_state=0)
our_model.fit(train_X, train_y)
prediction = our_model.predict(val_X)
mean_absolute_error(prediction, val_y)



1.7196969696969697

In [8]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)



In [17]:
for max_leaf_nodes in [5, 50, 500, 5000, 50000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print(f"Max leaf nodes: {max_leaf_nodes}  \t\t Mean Absolute Error: {my_mae}")

Max leaf nodes: 5  		 Mean Absolute Error: 1.2424242424242424
Max leaf nodes: 50  		 Mean Absolute Error: 1.553030303030303
Max leaf nodes: 500  		 Mean Absolute Error: 1.7651515151515151
Max leaf nodes: 5000  		 Mean Absolute Error: 1.7651515151515151
Max leaf nodes: 50000  		 Mean Absolute Error: 1.7651515151515151
Max leaf nodes: 500000  		 Mean Absolute Error: 1.7651515151515151


In [7]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(val_y, prediction)
print(cm)

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = our_model, X = train_X, y = train_y, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

[[ 0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2]
 [ 0  0  0  1  1  0  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  2  1]
 [ 0  0  0  0  0  0  0  0  2  4]
 [ 0  0  0  0  0  1  0  2  5 11]
 [ 0  1  1  0  1  0  2  2 17 17]
 [ 0  3  1  0  2  1  2 10 14 21]]
Accuracy: 32.69 %
Standard Deviation: 7.31 %


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(
